In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
import keras_tuner as kt 
import os
import json
import pandas as pd

%run preprocess.ipynb

Step 1: Splitting data...
Loading datasets...
Found 534 files belonging to 3 classes.
Found 90 files belonging to 3 classes.
Found 46 files belonging to 3 classes.

Classes found: ['1', '2', '3']
Class to PPB mapping: {0: 1.0, 1: 2.0, 2: 3.0}
Step 2: Converting labels to PPB values...
Step 3: Setting up data augmentation...
Step 4: Optimizing data pipeline...
Data preprocessing completed!


In [3]:
print("Step 6: Setting up training callbacks...")

# Training callbacks for better training control
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_mae',
        patience=15, #jumlah epoch yg ditunggu
        restore_best_weights=True, 
        verbose=1 #menampilkan pesan di konsol saat pelatihan dihentikan
    ), # Stop training if no improvement in validation loss
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5, #kurangi LR 50%
        patience=10,
        min_lr=1e-8,
        verbose=1
    ), # Reduce learning rate if no improvement
    tf.keras.callbacks.ModelCheckpoint(
        'best_aflatoxin_resnet50.keras',
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,
        verbose=1
    ) # Save best model
]

Step 6: Setting up training callbacks...


In [4]:
# Definisikan Global Variables yang akan digunakan di model.fit
INITIAL_EPOCHS = 30 
FINE_TUNE_EPOCHS = 35 
AUTOTUNE = tf.data.AUTOTUNE # Asumsi AUTOTUNE didefinisikan di preprocess.ipynb

In [5]:
# Fungsi Pembangunan Model untuk Keras Tuner (Menggantikan Kode Model Asli)

def build_model_for_tuning(hp):
    # --- 1. DEFENISI BASE MODEL ---
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    # 2. Setup FREEZE/UNFREEZE
    # TUNE: Fine-Tuning Depth (Lapisan Terdalam ResNet)
    fine_tune_at = hp.Choice('fine_tune_at', values=[170, 140]) 
    
    # Set semua trainable=True dulu, lalu bekukan yang awal
    base_model.trainable = True 
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
        
    # --- 3. PEMBANGUNAN HEAD LAYER ---
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    
    # TUNE: Jumlah Neuron di Lapisan Dense Pertama (L1) - Simplifikasi Model
    hp_units_l1 = hp.Choice('units_l1', values=[512, 128, 64])
    x = layers.Dense(units=hp_units_l1, activation='relu', name='hp_dense_1')(x)
    
    # TUNE: Dropout Rate
    hp_dropout = hp.Float('dropout_rate', min_value=0.2, max_value=0.4, step=0.1)
    x = layers.Dropout(rate=hp_dropout, name='hp_dropout')(x)
    
    # Tambahkan Dense Layer 2 (opsional, tetapi tetap kecil)
    x = layers.Dense(32, activation='relu', name='dense_32_fixed')(x) 
    
    # Lapisan Regresi Akhir (Tetap)
    outputs = layers.Dense(1, activation='linear', name='aflatoxin_output')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='HP_Estimator')

    # --- 4. KOMPILASI MODEL (Menggunakan LR Tuning) ---
    
    # TUNE: Learning Rate Phase 2
    hp_lr = hp.Choice('learning_rate', values=[1e-7, 1e-6, 5e-6]) 
    
    # TUNE: Optimizer
    hp_optimizer_name = hp.Choice('optimizer', values=['Adam', 'RMSprop'])
    
    if hp_optimizer_name == 'RMSprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=hp_lr)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=hp_lr)

    model.compile(
        optimizer=optimizer,
        loss='mse',
        metrics=['mae', 'mse']
    )
    
    return model

# Setup Model Checkpoint untuk Keras Tuner (Hanya simpan yang terbaik)
tuner_callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_mae', # Fokus pada MAE (lebih mudah diinterpretasikan)
        patience=10, 
        restore_best_weights=True,
        verbose=1
    ),
]

# Inisialisasi Keras Tuner (Random Search)
tuner = kt.RandomSearch(
    build_model_for_tuning,
    objective='val_mae', # Target: Meminimalkan Mean Absolute Error pada data validasi
    max_trials=100,       # Coba  kombinasi parameter yang berbeda
    executions_per_trial=1, # Tiap kombinasi dilatih 1 kali
    directory='aflatoxin_tuning_results',
    project_name='ResNet_Aflatoxin_Reg3'
)

In [6]:
# Cell 3 & 4 (DIGABUNG dan DIUBAH total)

# 1. Pastikan model dengan LR 1e-6 sudah dilatih dan disimpan
# ASUMSI: File 'best_aflatoxin_resnet50.keras' berisi bobot terbaik Fase 1 (Frozen Base)

# 2. Muat Bobot Terbaik FASE 1
# Muat model terbaik Fase 1 (Frozen Base) untuk dijadikan bobot awal pada semua trial RS.
# Ini penting karena RS harus mulai dari titik terbaik yang sudah stabil.
best_frozen_model = tf.keras.models.load_model('best_aflatoxin_resnet50.keras')
initial_weights = best_frozen_model.get_weights()


print("Step 7: Starting Random Search for Fine-Tuning Hyperparameters...")

# --- JALANKAN SEARCH ---
# Kita menjalankan RS dengan epochs total Phase 2 (Fine-Tuning)
tuner.search(
    train_ds,
    validation_data=val_ds,
    epochs=FINE_TUNE_EPOCHS, 
    callbacks=tuner_callbacks,
    # Menggunakan Initial Weights dari Fase 1 untuk semua trial:
    # Keras Tuner akan menggunakan weights ini untuk inisialisasi pada build_model_for_tuning
    # Namun, karena tidak ada argumen `initial_weights` di tuner.search, 
    # kita harus memodifikasi fungsi build_model_for_tuning untuk memuatnya secara eksplisit
    # (Ini membutuhkan penyesuaian yang kompleks, jadi kita fokus pada tuning LR/Dropout/Unit)
    verbose=1
)

print("\nStep 8: Getting the best model...")
# Ambil model terbaik berdasarkan MAE Validasi
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]

print("\n--- HASIL HYPERPARAMETER TERBAIK ---")
print(best_hp.values)

Trial 93 Complete [00h 03m 07s]
val_mae: 1.2922585010528564

Best val_mae So Far: 0.5566709041595459
Total elapsed time: 13h 39m 41s

Step 8: Getting the best model...

--- HASIL HYPERPARAMETER TERBAIK ---
{'fine_tune_at': 170, 'units_l1': 512, 'dropout_rate': 0.2, 'learning_rate': 5e-06, 'optimizer': 'Adam'}


c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [7]:
# Cell 9 (DIGANTI TOTAL, Menjadi Post-Tuning Analysis)

print("Step 9: Analyzing Random Search Results...")

# Ambil model terbaik berdasarkan MAE Validasi (setelah Random Search selesai)
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]

print("\n--- HYPERPARAMETER TERBAIK DITEMUKAN ---")
print(f"LR: {best_hp.get('learning_rate'):.1e}")
print(f"Dropout Rate: {best_hp.get('dropout_rate'):.1f}")
print(f"Units L1: {best_hp.get('units_l1')}")
print(f"Fine Tune At: {best_hp.get('fine_tune_at')}")

# Jika Anda ingin melanjutkan pelatihan dengan HP terbaik ini (Fase 2)
# Anda harus memuat bobot Phase 1 terbaik dan melatih ulang dengan HP yang ditemukan.

# model = best_model
# print("Model sekarang menggunakan HP terbaik dari Random Search.")

Step 9: Analyzing Random Search Results...

--- HYPERPARAMETER TERBAIK DITEMUKAN ---
LR: 5.0e-06
Dropout Rate: 0.2
Units L1: 512
Fine Tune At: 170


In [8]:
print("Step 9a: Generating Comparison Table for All Random Search Trials...")

# --- Ambil semua trial yang telah dijalankan ---
# Gunakan tuner.oracle.trials.values() untuk mendapatkan semua objek trial
all_trials = tuner.oracle.trials.values()
trial_data = []

# Ekstrak Hyperparameter dan Skor dari setiap trial
for trial in all_trials:
    # Lanjutkan hanya jika trial sudah selesai (status 'COMPLETED')
    if trial.status == 'COMPLETED':
        trial_id = trial.trial_id
        hp = trial.hyperparameters.values
        
        # Ambil skor terbaik (val_MAE) langsung dari objek trial
        # Skor ini adalah nilai terbaik yang dicapai selama trial tersebut
        score = trial.score 
        
        # Kumpulkan data ke dalam dictionary
        trial_record = {
            'Trial ID': trial_id,
            'val_MAE': score,
            'LR (Phase 2)': hp.get('learning_rate'),
            'Optimizer': hp.get('optimizer'),
            'Units L1': hp.get('units_l1'),
            'Dropout': hp.get('dropout_rate'),
            'Fine Tune At': hp.get('fine_tune_at')
        }
        trial_data.append(trial_record)

# --- Konversi ke DataFrame dan Tampilkan ---
if trial_data:
    df_results = pd.DataFrame(trial_data)

    # Urutkan berdasarkan val_MAE (semakin kecil semakin baik)
    df_results = df_results.sort_values(by='val_MAE', ascending=True)

    # Format kolom numerik untuk tampilan yang lebih rapi
    df_results['val_MAE'] = df_results['val_MAE'].round(4)
    df_results['LR (Phase 2)'] = df_results['LR (Phase 2)'].apply(lambda x: f'{x:.1e}')
    df_results['Dropout'] = df_results['Dropout'].round(2)

    print("\n--- PERBANDINGAN HASIL SEMUA TRIAL RANDOM SEARCH (TERBAIK KE TERBURUK) ---")
    # Reset index untuk tampilan ranking
    df_results = df_results.reset_index(drop=True)
    df_results.index.name = 'Rank'
    df_results.index += 1 # Mulai dari Rank 1

    # Tampilkan tabel dalam format markdown
    print(df_results.to_markdown())
else:
    print("\nTidak ada trial yang selesai untuk ditampilkan.")

Step 9a: Generating Comparison Table for All Random Search Trials...

--- PERBANDINGAN HASIL SEMUA TRIAL RANDOM SEARCH (TERBAIK KE TERBURUK) ---
|   Rank |   Trial ID |   val_MAE |   LR (Phase 2) | Optimizer   |   Units L1 |   Dropout |   Fine Tune At |
|-------:|-----------:|----------:|---------------:|:------------|-----------:|----------:|---------------:|
|      1 |        085 |    0.5567 |          5e-06 | Adam        |        512 |       0.2 |            170 |
|      2 |        083 |    0.5573 |          1e-06 | RMSprop     |        128 |       0.3 |            170 |
|      3 |        057 |    0.5574 |          5e-06 | Adam        |        512 |       0.4 |            170 |
|      4 |        039 |    0.5575 |          1e-06 | Adam        |        512 |       0.4 |            170 |
|      5 |        068 |    0.5579 |          5e-06 | RMSprop     |        128 |       0.4 |            170 |
|      6 |        080 |    0.5581 |          5e-06 | Adam        |        128 |       0.3 | 

In [9]:
def combine_keras_tuner_results(main_directory):
    """
    Membaca semua hasil trial dari beberapa proyek Keras Tuner,
    menggabungkannya, dan menghapus duplikat.
    """
    all_trials_data = []
    
    # Memeriksa apakah direktori utama ada
    if not os.path.exists(main_directory):
        print(f"Error: Direktori '{main_directory}' tidak ditemukan.")
        return None

    # Menelusuri setiap subdirektori (proyek) di dalam direktori utama
    for project_name in os.listdir(main_directory):
        project_path = os.path.join(main_directory, project_name)
        
        # Pastikan itu adalah sebuah direktori
        if os.path.isdir(project_path):
            print(f"Membaca proyek: {project_name}...")
            
            # Menelusuri setiap folder trial di dalam proyek
            for trial_dir in os.listdir(project_path):
                trial_path = os.path.join(project_path, trial_dir)
                trial_json_path = os.path.join(trial_path, 'trial.json')
                
                if os.path.isfile(trial_json_path):
                    try:
                        with open(trial_json_path, 'r') as f:
                            data = json.load(f)
                        
                        # Hanya proses trial yang sudah selesai
                        if data.get('status') == 'COMPLETED':
                            hp = data.get('hyperparameters', {}).get('values', {})
                            score = data.get('score')

                            # Kumpulkan data jika trial valid
                            if score is not None and hp:
                                trial_record = {
                                    'Project': project_name,
                                    'Trial ID': data.get('trial_id'),
                                    'val_MAE': score,
                                    'LR (Phase 2)': hp.get('learning_rate'),
                                    'Optimizer': hp.get('optimizer'),
                                    'Units L1': hp.get('units_l1'),
                                    'Dropout': hp.get('dropout_rate'),
                                    'Fine Tune At': hp.get('fine_tune_at')
                                }
                                all_trials_data.append(trial_record)
                                
                    except (json.JSONDecodeError, KeyError) as e:
                        print(f"  - Gagal membaca atau memproses {trial_json_path}: {e}")

    if not all_trials_data:
        print("Tidak ada data trial yang berhasil dibaca.")
        return None

    # Konversi ke DataFrame
    df_combined = pd.DataFrame(all_trials_data)
    
    print(f"\nTotal trial yang berhasil dibaca sebelum penghapusan duplikat: {len(df_combined)}")

    # Menentukan kolom untuk memeriksa duplikasi
    # Kita ingin menghapus baris di mana kombinasi parameter DAN hasilnya sama persis
    duplicate_check_columns = [
        'val_MAE', 'LR (Phase 2)', 'Optimizer', 
        'Units L1', 'Dropout', 'Fine Tune At'
    ]
    
    # Menghapus duplikat
    df_cleaned = df_combined.drop_duplicates(subset=duplicate_check_columns, keep='first')
    
    print(f"Total trial setelah menghapus duplikat: {len(df_cleaned)}")
    
    # Mengurutkan berdasarkan metrik terbaik (val_MAE terendah)
    df_sorted = df_cleaned.sort_values(by='val_MAE', ascending=True)

    # Merapikan format untuk presentasi
    df_sorted['val_MAE'] = df_sorted['val_MAE'].round(4)
    df_sorted['LR (Phase 2)'] = df_sorted['LR (Phase 2)'].apply(lambda x: f'{x:.0e}' if x is not None else 'N/A')
    df_sorted['Dropout'] = df_sorted['Dropout'].round(2)

    # Mereset index untuk ranking
    df_sorted = df_sorted.reset_index(drop=True)
    df_sorted.index.name = 'Rank'
    df_sorted.index += 1
    
    return df_sorted

# --- PENGGUNAAN ---
# Tentukan direktori utama tempat semua proyek tuning Anda disimpan
tuning_directory = 'aflatoxin_tuning_results' 
final_results_df = combine_keras_tuner_results(tuning_directory)

# Tampilkan hasil akhir dalam format yang rapi
if final_results_df is not None:
    print("\n--- HASIL GABUNGAN SEMUA PERCOBAAN RANDOM SEARCH (SETELAH PEMBERSIHAN) ---")
    print(final_results_df.to_markdown())

Membaca proyek: ResNet_Aflatoxin_Reg...
Membaca proyek: ResNet_Aflatoxin_Reg2...
Membaca proyek: ResNet_Aflatoxin_Reg3...

Total trial yang berhasil dibaca sebelum penghapusan duplikat: 290
Total trial setelah menghapus duplikat: 290

--- HASIL GABUNGAN SEMUA PERCOBAAN RANDOM SEARCH (SETELAH PEMBERSIHAN) ---
|   Rank | Project               |   Trial ID |   val_MAE |   LR (Phase 2) | Optimizer   |   Units L1 |   Dropout |   Fine Tune At |
|-------:|:----------------------|-----------:|----------:|---------------:|:------------|-----------:|----------:|---------------:|
|      1 | ResNet_Aflatoxin_Reg  |         29 |    0.5539 |          5e-06 | RMSprop     |        128 |       0.3 |            140 |
|      2 | ResNet_Aflatoxin_Reg3 |        085 |    0.5567 |          5e-06 | Adam        |        512 |       0.2 |            170 |
|      3 | ResNet_Aflatoxin_Reg3 |        083 |    0.5573 |          1e-06 | RMSprop     |        128 |       0.3 |            170 |
|      4 | ResNet_Aflatox